In [32]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split,GridSearchCV
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Dropout


(x_train, y_train),(x_test,y_test)= mnist.load_data()

#hata verip vermediğini kontrol etmek için veri setini küçülttüm
x_train=x_train[:10]
x_test=x_test[:5]
y_train = y_train[:10]
y_test = y_test[:5]

#siyah beyaz ise 1 rgb ise 3 katmanlı ancak 1 eksik reshape ile girdi katman sayısını belirttim
x_train= x_train.reshape(60000,28,28,1)
x_test= x_test.reshape(10000,28,28,1)

#piksel yapısını matris içerisinde sayı olarak barındırabilmem için farklı tipe çevirmem gerekiyor
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

#normalizasyon
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

#to_categorical ile encoding işlemi yaptım
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=0)

def modelkur(filtr=32,kernel=3,conv_layer=3,dense_layer=2,units=32,learning_rate=0.01,dropout_rate=0.2):

  model = Sequential()
  model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(28,28,1),activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))

  for i in range(conv_layer):
    model.add(Conv2D(filters=filtre,kernel_size=(kernel,kernel),activation="relu",padding="same"))
    model.add(MaxPooling2D(pool_size=(2,2),padding="same "))

  model.add(Flatten())
  model.add(Dense(32,activation="relu"))

  for j in range(dense_layer):
    model.add(Dense(units,activation="relu"))
    model.add(Dropout(dropout_rate))

    model.add(Dense(10,activation="softmax"))
    model.compile(optimizer=Adam(learning_rate),loss="categorical_crossentropy",metrics=["accuracy"])

  return model
# Modeli en iyi parametrelerle eğitin
parametreler={
     "filtreler":[32,64],
     "kernel":[3,5],
     "conv_layer":[3,5],
     "dense_layer":[2,3],
     "units":[32,64],
     "learning_rate":[0.01,0.001],
     "dropout_rate":[0.2,0.3],
     "epochs": [20,30]
 }

sınıf = KerasClassifier(build_fn=modelkur,verbose=0)
grid =GridSearchCV(estimator=sınıf,param_grid=parametreler,cv=3)
gridsonuc = grid.fit(x_train,y_train)
bestparam = gridsonuc.best_params_

bestmodel=modelkur(filtre=bestparam["filtre"],kernel=bestparam["kernel"],conv_layer=bestparam["conv_layer"],dense_layer=bestparam["dense_layer"],units=bestparam["units"],learning_rate=bestparam["learning_rate"],dropout_rate=bestparam["dropout_rate"])
# Modeli eğitmeye devam et
cıktı = bestmodel.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=128, verbose=0)



fig,ax=plt.subplots(1,2,figsize=(25,10))
# Loss grafiği
ax[0].plot(cıktı.history["loss"],color="blue",label="train loss")
ax[0].plot(cıktı.history["val_loss"],color="orange",label="validation loss")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend()
ax[0].set_title("Loss Graph")
# Accuracy grafiği
ax[1].plot(cıktı.history["accuracy"],color="blue",label="train accuracy")
ax[1].plot(cıktı.history["val_accuracy"],color="orange",label="validation accuracy")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("accuarcy")
ax[1].legend()
ax[1].set_title("accuarcy graph")

plt.show()

ValueError: Invalid parameter conv_layer for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(conv_layer=3)`
Check the list of available parameters with `estimator.get_params().keys()`